## 1. 테스트 테이블 생성 (SQL)

테스트용 테이블을 생성합니다. 이미 존재하는 경우 스킵합니다.

In [ ]:
CREATE TABLE IF NOT EXISTS test_table (
    id INTEGER AUTOINCREMENT,
    timestamp TIMESTAMP,
    value FLOAT,
    category VARCHAR(50)
);

## 2. 랜덤 데이터 생성 (Python)

현재 시간과 랜덤한 값, 카테고리를 생성합니다.

In [ ]:
import random
from datetime import datetime

v_current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
random_value = random.uniform(0, 100)
categories = ['A', 'B', 'C', 'D']
random_category = random.choice(categories)

print(f"Time: {v_current_time}")
print(f"Value: {random_value}")
print(f"Category: {random_category}")

## 3. 데이터 삽입 (SQL)

이전 셀에서 생성한 Python 변수를 Jinja 템플릿으로 참조하여 테이블에 데이터를 삽입합니다.

In [ ]:
INSERT INTO test_table (timestamp, value, category)
VALUES ('{{v_current_time}}', {{random_value}}, '{{random_category}}')

## 4. 데이터 조회 (SQL)

입력된 데이터를 SQL 쿼리로 조회합니다.

In [ ]:
SELECT * FROM test_table ORDER BY timestamp DESC

## 5. 데이터 시각화 (Python with Snowpark)

Snowpark DataFrame을 사용하여 집계 연산을 수행하고 차트로 시각화합니다.

_이 셀을 실행하려면 **matplotlib**와 **seaborn**패키지를 추가해야합니다_

- 화면 상단의 Packages를 엽니다
- Find Packages에서 **matplotlib**와 **seaborn**을 각각 검색합니다
- `save` 버튼을 클릭합니다
- notebook 세션이 다시 Active 상태가 될 때까지 기다립니다

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, count, avg, min as sp_min, max as sp_max, stddev

# Get active Snowpark session
session = get_active_session()

# 테이블에서 직접 Snowpark DataFrame 생성
df_snowpark = session.table("test_table").sort(col("TIMESTAMP").desc())

# Snowpark DataFrame으로 집계 연산 수행
# 1. Category별 Count
df_category_counts = df_snowpark.group_by(col("CATEGORY")) \
    .agg(count("*").alias("COUNT")) \
    .sort(col("CATEGORY"))

# 2. Category별 Average
df_category_avg = df_snowpark.group_by(col("CATEGORY")) \
    .agg(avg(col("VALUE")).alias("AVG_VALUE")) \
    .sort(col("CATEGORY"))

# 3. Category별 Minimum
df_category_min = df_snowpark.group_by(col("CATEGORY")) \
    .agg(sp_min(col("VALUE")).alias("MIN_VALUE")) \
    .sort(col("CATEGORY"))

# 4. Category별 Maximum
df_category_max = df_snowpark.group_by(col("CATEGORY")) \
    .agg(sp_max(col("VALUE")).alias("MAX_VALUE")) \
    .sort(col("CATEGORY"))

# 전체 통계 (Snowpark로 계산)
df_stats = df_snowpark.agg(
    count("*").alias("TOTAL_COUNT"),
    avg(col("VALUE")).alias("MEAN_VALUE"),
    sp_min(col("VALUE")).alias("MIN_VALUE"),
    sp_max(col("VALUE")).alias("MAX_VALUE"),
    stddev(col("VALUE")).alias("STDDEV_VALUE")
)

# Category별 전체 통계 (Snowpark로 계산)
df_category_stats = df_snowpark.group_by(col("CATEGORY")) \
    .agg(
        count("*").alias("COUNT"),
        avg(col("VALUE")).alias("AVERAGE"),
        sp_min(col("VALUE")).alias("MINIMUM"),
        sp_max(col("VALUE")).alias("MAXIMUM")
    ) \
    .sort(col("CATEGORY"))

# 시각화를 위해 집계 결과만 pandas로 변환 (원본 데이터가 아닌 집계 결과)
category_counts_pd = df_category_counts.to_pandas()
category_avg_pd = df_category_avg.to_pandas()
category_min_pd = df_category_min.to_pandas()
category_max_pd = df_category_max.to_pandas()
stats_pd = df_stats.to_pandas()
category_stats_pd = df_category_stats.to_pandas()

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.subplots_adjust(hspace=0.4, wspace=0.3)

# Color palette
colors = sns.color_palette('Set2', len(category_counts_pd))

# 1. Count by Category
bars1 = axes[0, 0].bar(category_counts_pd['CATEGORY'], category_counts_pd['COUNT'], 
                       color=colors, edgecolor='black', linewidth=2, width=0.6)
for bar in bars1:
    height = bar.get_height()
    axes[0, 0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}', ha='center', va='bottom', 
                    fontweight='bold', fontsize=11)
axes[0, 0].set_xlabel('Category', fontsize=13, fontweight='bold')
axes[0, 0].set_ylabel('Count', fontsize=13, fontweight='bold')
axes[0, 0].set_title('Count by Category', fontsize=15, fontweight='bold', pad=20)
axes[0, 0].grid(True, alpha=0.3, axis='y', linestyle='--')

# 2. Average by Category
bars2 = axes[0, 1].bar(category_avg_pd['CATEGORY'], category_avg_pd['AVG_VALUE'],
                       color=colors, edgecolor='black', linewidth=2, width=0.6)
for bar in bars2:
    height = bar.get_height()
    axes[0, 1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}', ha='center', va='bottom', 
                    fontweight='bold', fontsize=11)
axes[0, 1].set_xlabel('Category', fontsize=13, fontweight='bold')
axes[0, 1].set_ylabel('Average Value', fontsize=13, fontweight='bold')
axes[0, 1].set_title('Average by Category', fontsize=15, fontweight='bold', pad=20)
axes[0, 1].grid(True, alpha=0.3, axis='y', linestyle='--')

# 3. Minimum by Category
bars3 = axes[1, 0].bar(category_min_pd['CATEGORY'], category_min_pd['MIN_VALUE'],
                       color=colors, edgecolor='black', linewidth=2, width=0.6)
for bar in bars3:
    height = bar.get_height()
    axes[1, 0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}', ha='center', va='bottom', 
                    fontweight='bold', fontsize=11)
axes[1, 0].set_xlabel('Category', fontsize=13, fontweight='bold')
axes[1, 0].set_ylabel('Min Value', fontsize=13, fontweight='bold')
axes[1, 0].set_title('Minimum by Category', fontsize=15, fontweight='bold', pad=20)
axes[1, 0].grid(True, alpha=0.3, axis='y', linestyle='--')

# 4. Maximum by Category
bars4 = axes[1, 1].bar(category_max_pd['CATEGORY'], category_max_pd['MAX_VALUE'],
                       color=colors, edgecolor='black', linewidth=2, width=0.6)
for bar in bars4:
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}', ha='center', va='bottom', 
                    fontweight='bold', fontsize=11)
axes[1, 1].set_xlabel('Category', fontsize=13, fontweight='bold')
axes[1, 1].set_ylabel('Max Value', fontsize=13, fontweight='bold')
axes[1, 1].set_title('Maximum by Category', fontsize=15, fontweight='bold', pad=20)
axes[1, 1].grid(True, alpha=0.3, axis='y', linestyle='--')

plt.show()

# Summary statistics (Snowpark로 계산된 결과 출력)
print("=" * 60)
print("Data Summary".center(60))
print("=" * 60)
print(f"Total Records: {int(stats_pd['TOTAL_COUNT'].iloc[0]):,}")
print(f"Value Range: {stats_pd['MIN_VALUE'].iloc[0]:.2f} ~ {stats_pd['MAX_VALUE'].iloc[0]:.2f}")
print(f"Mean Value: {stats_pd['MEAN_VALUE'].iloc[0]:.2f}")
print(f"Standard Deviation: {stats_pd['STDDEV_VALUE'].iloc[0]:.2f}")

print("\n" + "=" * 60)
print("Category Statistics".center(60))
print("=" * 60)
category_stats_pd.columns = ['Category', 'Count', 'Average', 'Minimum', 'Maximum']
category_stats_pd = category_stats_pd.set_index('Category')
print(category_stats_pd.round(2).to_string())